# Tutorial on the Few-shot RAW Image Denoising Track of the MIPI Workshop
\[[Homepage](https://mipi-challenge.org/MIPI2024/)\] \[[Codalab](https://codalab.lisn.upsaclay.fr/competitions/17017)\]

In this tutorail, we will go through the steps of the RAW image denoising task, including:

- **Dataset**: How to read and load paired data from the dataset.
- **Lite ISP for processing the RAW image**: How to visualize the processed raw images.
- **Score Calculation**: Get quantitative metrics to evaluate performance.
- **Submission**: How to create a zip file that complies with submission format.

In [ ]:
import sys
import os
## add current path to the enviorment
sys.path.append(os.path.abspath(os.path.dirname(__file__)))

## Dataset

> Source code can be found in `tools/mipi_starting_kit/code_example/dataset.py`.


In [ ]:
## The dataset can be also found in `led/data/mipi_dataset.py`
## In this way you could use our codebase for both training and evaluation.
from dataset import MIPIDataset

opt = {
    ## Basic options of the dataset
    'phase': 'train',                           # the phase: train or valid
    'dataroot': 'path/to/your/dataset',         # the path to our released dataset
    'data_pair_list': 'path/to/your/pair/list', # the path to the txt file of the mipi dataset
    ## Augmentation options
    'crop_size': 1024,                          # whether to crop the paired data (center crop when not in train phase)
    'use_hflip': True,                          # whether to use the flip augmentation
    'use_rot':   True,                          # whether to use the rotation augmentation
}

dataset = MIPIDataset(opt)

for paired_data in dataset:
    """
    Operate with the data, the `paired_data` are composed as below:
    {
        'lq': lq_im_patch,      # the input patch in (4, crop_size, crop_size). torch.Tensor
        'gt': gt_im_patch,      # the gt patch, same size with lq. torch.Tensor
        'ratio': ratio,         # the value of the additional dgain. torch.Tensor
        'wb': gt_wb,            # the white balance of gt in (3, 1, 1). torch.Tensor
        'ccm': gt_ccm,          # the cam2rgb matrix in (3, 3). torch.Tensor
        'lq_path': lq_path,     # the path to the input npz file. str
        'gt_path': gt_path      # the path to the gt npz file. str
    }
    """
    pass

## Lite ISP
> We utilze a simplified version of the [ELD](https://github.com/Vandermode/ELD). 
>
> Source code can be found in `tools/mipi_starting_kit/code_example/lite_isp.py`

In [ ]:
from dataset import depack_meta, calculate_ratio    # for load data
from lite_isp import process

## load data
example_meta_path = 'path/to/the/npz/file'
raw, wb, cam2rgb = depack_meta(example_meta_path, to_tensor=True)

## amplified the raw image
ratio = calculate_ratio(example_meta_path)
amplified_raw = (raw * ratio).clamp(0.0, 1.0)

## prepare for process
## the process function only support batch format, which is (B, 4, H, W)
batch_raw = amplified_raw.unsqueeze(0)
wbs       = wb.unsqueeze(0)
cam2rgbs  = cam2rgb.unsqueeze(0)

## process!
rgb = process(batch_raw, wbs, cam2rgbs, gamma=2.2)

## save the image
from torchvision.utils import save_image
# we save the gt image with the `save_image` function provided by torchvision too
save_image(rgb, 'path/to/save/the/image.png')

## Score Calculation

> Since raw images take up a lot of storage space, and codalab only supports uploading a zip packages of up to 300MB. we calculate the metric on 8-bit pngs (in sRGB color space). 
> 
> And the 8bit png data in sRGB space **MUST** be rendered by the [Lite ISP](#lite-isp) we provide. We will double check the contestants' codes at the end to ensure fairness.

The score is calculated with both PSNR and SSIM.

$$Score=\log_k(SSIM*k^{PSNR})=PSNR+\log_k(SSIM)$$

In our implementation, $k=1.2$.

In [ ]:
from skimage import io
from score import calculate_score

## load images
# We utilize the io.imread from skimage too on the codalab server.
result = io.imread('path/to/the/result.png')
gt     = io.imread('path/to/the/gt.png')

## calculate the score
score = calculate_score(result, gt)

## Submission

The validation/test data are provided in the following format:
```bash
|-- test
|   |-- Camera1
|   |   `-- short
|   |       |-- 00002_05_33.33333333333333.npz
|   |       `-- ...
|   `-- Camera2
|       `-- short
|           |-- 00002_25_33.33333333333333.npz
|           `-- ...
`-- valid
    |-- Camera1
    |   `-- short
    |       |-- 00001_05_33.33333333333333.npz
    |       `-- ...
    `-- Camera2
        `-- short
            |-- 00001_25_33.33333333333333.npz
            `-- ...
```

And as your submission, you should upload a zip file which are as the following format:

```bash
|-- test.zip
|   |-- Camera1
|   |   |-- 00002_05_33.33333333333333.png
|   |   `-- ...
|   `-- Camera2
|       |-- 00002_25_33.33333333333333.png
|       `-- ...
`-- valid.zip
    |-- Camera1
    |   |-- 00001_05_33.33333333333333.png
    |       `-- ...
    `-- Camera2
        |-- 00001_25_33.33333333333333.png
        `-- ...
```

Here are a few points to pay special attention to:
- Different zip (test.zip/valid.zip) should be upload to different phase of our challenge.
- There is no longer a subfolder `short` under `Camera1` or `Camera2`
- The output image should be the same name with the input (except the suffix).
- The uploaded image should be 8-bit png images, and **MUST** be rendered by the [Lite ISP](#lite-isp) we provide.


In [ ]:
""" A simple demo for output the image """
import os
from glob import glob
from dataset import calculate_ratio, depack_meta
from lite_isp import process
from torchvision.utils import save_image

cameras = ['Camera1', 'Camera2']
data_path = 'path/to/the/valid/data'
save_path = 'path/to/save'

my_algo = lambda x, camera: x       # a function for process the image
                                    # you could use different weights for different cameras

for curr_cam in cameras:
    os.makedirs(f'{save_path}/{curr_cam}', exist_ok=True)
    input_npzs = list(sorted(glob(f'{data_path}/*.npz')))
    for input_npz in input_npzs:
        ## load and prepare data
        ratio = calculate_ratio(input_npz)
        im, wb, cam2rgb = depack_meta(input_npz, to_tensor=True)
        im = (im * ratio).clamp(None, 1.0).unsqueeze(0)
        wb = wb.unsqueeze(0)
        cam2rgb = cam2rgb.unsqueeze(0)

        ## denoising
        out = my_algo(im, curr_cam)

        ## perform lite isp and save image
        rgb = process(out, wb, cam2rgb, gamma=2.2)

        ## save image
        curr_im_name = os.path.basename(input_npz).replace('.npz', '.png')
        save_image(rgb, f'{save_path}/{curr_cam}/{curr_im_name}')
